In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
loader = PyPDFLoader('resume3.pdf')
content = loader.load()
content

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-11-03T18:36:25+05:30', 'title': 'Sample resume - student', 'author': 'Jenon', 'moddate': '2025-11-03T18:36:25+05:30', 'source': 'resume3.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="Hayden Smith   \n214 Mitre Avenue, Park Hill, 3045 |  04501 123 456  |  haydensmith@email.com  \n  \n(Tip: Name, mobile number and email address are essential. Current address could be included \nespecially if you live nearby. Ensure that it is clearly displayed)  \nhttps://leetcode.com/u/HLHWTbXcl7/ \nhttps://www.naukri.com/code360/profile/30fc2a8a-aace-4817-8f6c-6f867680fbe4    \nCareer Objective   \nI am reliable hard working Year 11 student seeking casual or part-time customer service work in a \nsports retail environment. Having played soccer for nine years and a keen all-round sports \nenthusiast, I am looking to contribute knowledge and proven communications skills.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap = 20)
chunks = splitter.split_documents(content)
chunks

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-11-03T18:36:25+05:30', 'title': 'Sample resume - student', 'author': 'Jenon', 'moddate': '2025-11-03T18:36:25+05:30', 'source': 'resume3.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Hayden Smith   \n214 Mitre Avenue, Park Hill, 3045 |  04501 123 456  |  haydensmith@email.com  \n  \n(Tip: Name, mobile number and email address are essential. Current address could be included \nespecially if you live nearby. Ensure that it is clearly displayed)  \nhttps://leetcode.com/u/HLHWTbXcl7/ \nhttps://www.naukri.com/code360/profile/30fc2a8a-aace-4817-8f6c-6f867680fbe4    \nCareer Objective   \nI am reliable hard working Year 11 student seeking casual or part-time customer service work in a'),
 Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-11-03T18:36:25+05:30', 'title': 'Sample resume - student', 'a

In [4]:
resume_content = ''
for i in range(len(content)):
    resume_content += content[i].page_content
resume_content

"Hayden Smith   \n214 Mitre Avenue, Park Hill, 3045 |  04501 123 456  |  haydensmith@email.com  \n  \n(Tip: Name, mobile number and email address are essential. Current address could be included \nespecially if you live nearby. Ensure that it is clearly displayed)  \nhttps://leetcode.com/u/HLHWTbXcl7/ \nhttps://www.naukri.com/code360/profile/30fc2a8a-aace-4817-8f6c-6f867680fbe4    \nCareer Objective   \nI am reliable hard working Year 11 student seeking casual or part-time customer service work in a \nsports retail environment. Having played soccer for nine years and a keen all-round sports \nenthusiast, I am looking to contribute knowledge and proven communications skills.   \n  \n(Tip: A career objective isn’t essential, but it’s useful if you don't have much experience and can \nconvey enthusiasm and motivation. Briefly summarise any work you have done, your strengths and \nrelevant expertise and state how you aim to apply this to your career goal. Adjust the statement to \nreflect 

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
load_dotenv()
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash',api_key=os.getenv('GEMINI_API_KEY'))

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM RESUME:
        {resume_data}
        ### INSTRUCTION:
        The scraped text is from the Resume of a candidate looking for jobs.
        Your job is to extract the content of this resume and return them in JSON format containing the 
        following keys: `certifications`, `issuing authority`, `date earned`, `certificate links` and `coding platforms profile links` .
        Only return the valid JSON. Store multiple records of a key in form of list.
        ### VALID JSON (NO PREAMBLE):    
        """
)
chain = prompt_extract | llm | JsonOutputParser()
chain.invoke(resume_content)

{'certifications': ['Introduction to GenAI',
  'Introduction to Prompt Engineering'],
 'issuing authority': ['Google Cloud', 'NVIDIA'],
 'date earned': [],
 'certificate links': [],
 'coding platforms profile links': ['https://leetcode.com/u/HLHWTbXcl/',
  'https://www.naukri.com/code360/profile/30fc2a8a-aace-4817-8f6c-6f867680fbe4']}

In [28]:
analyser_prompt = PromptTemplate.from_template("""You are a proffesional HR manager who is experienced in reviewing resumes, analyse the resume content given below and provide the candidate some personalized improvement recommendations (e.g., “work on SQL basics,”
“improve communication skills”).
                                Resume Content:
                                               {resume_content}""")

In [35]:
analyser_prompt = PromptTemplate.from_template('''You are a senior technical recruiter, skilled at 10-second resume reviews.

Analyze the resume below, which is for a candidate targeting a technical role.

Provide only one  concise, **one-liner** improvement recommendations for the candidate. These should be short, actionable tips, exactly like the examples below.

Examples of desired format:
* “Quantify your project achievements.”
* “Add a dedicated 'Technical Skills' section.”
* “Tailor your summary to the job description.”
* “Improve focus on SQL and data visualization.”

Resume Content:
{resume_content}''')

In [36]:
from langchain_core.output_parsers import StrOutputParser
analyser_chain = analyser_prompt | llm | StrOutputParser()

In [37]:
analyser_chain.invoke(resume_content)

'Align your career objective with technical aspirations.'

# Generate Interview questions

In [18]:
interview_prompt = PromptTemplate.from_template('''
    Persona: You are an expert technical interviewer with over a decade of experience hiring for data science and analytics teams. You specialize in evaluating entry-level (fresher) talent.

Context: Your goal is to create a robust interview question bank for the "Junior Data Scientist" role. The questions must be appropriate for a candidate with primarily academic and project-based experience (a fresher).

Task: Refer to the job description provided below. Generate a comprehensive list of interview questions.

Requirements:

Structure: Present the output as a clearly structured json consisting of interview questions stored in a python list.
Please structure your response as a single JSON object.

The root object must contain a single key named interview_questions.

The value of interview_questions must be a list of objects.

Each object in that list must have exactly two keys:

"category": A string describing the topic (e.g., "SQL & Database Concepts").

"questions": A list of strings, where each string is a single interview question.
Job Description:

{job_desc}''')

In [19]:
job_desc='''Job Title: Junior Data Scientist
Location: [City, State] Position Type: Full-Time, Entry-Level

Company Overview
[Brief, 1-2 sentence description of your company, its mission, and the problems you are solving. Example: "At [Company Name], we are a fast-growing tech firm dedicated to revolutionizing [Your Industry] through data-driven insights. We believe in the power of data to shape products, strategies, and user experiences."]

Job Summary
We are looking for a motivated and curious Junior Data Scientist to join our dynamic analytics team. This is an ideal role for a recent graduate or early-career professional passionate about turning large datasets into actionable insights. You will work closely with senior data scientists, engineers, and product managers to tackle challenging problems, from data collection and cleaning to building and deploying machine learning models.

Core Responsibilities
Data Collection & Cleaning: Assist in gathering data from various sources (SQL databases, APIs, flat files) and perform necessary preprocessing and cleaning to ensure data quality and integrity.

Exploratory Data Analysis (EDA): Analyze data to identify trends, patterns, and anomalies. Use statistical methods to uncover insights and formulate hypotheses.

Model Building & Validation: Apply fundamental machine learning algorithms (e.g., regression, classification, clustering) to build predictive models. You will be responsible for training, testing, and validating these models.

Data Visualization: Create compelling reports and dashboards using tools like Tableau, Power BI, or Python libraries (Matplotlib, Seaborn) to communicate your findings to both technical and non-technical stakeholders.

Collaboration: Work collaboratively with cross-functional teams to understand business requirements and deliver data-driven solutions that meet their needs.

Qualifications & Skills
Required:
Educational Background: A Bachelor's or Master's degree in Data Science, Computer Science, Statistics, Mathematics, Economics, or another related quantitative field.

Technical Skills:

Strong proficiency in Python and its data science ecosystem (pandas, NumPy, scikit-learn).

Solid understanding of SQL for data querying and manipulation.

Familiarity with data visualization libraries (e.g., Matplotlib, Seaborn) or BI tools (e.g., Tableau, Power BI).

Knowledge Base:

A firm grasp of core statistical concepts and machine learning fundamentals (supervised vs. unsupervised learning, model evaluation metrics, etc.).

Soft Skills:

Excellent problem-solving and analytical abilities.

Strong communication skills, with the ability to explain complex technical concepts to a non-technical audience.

A natural curiosity and a strong desire to learn.

Preferred (Nice to Have):
Prior internship experience in a data-related role (data analyst, data scientist, etc.).

A portfolio of data science projects (e.g., on GitHub) that showcases your skills and passion.

Experience with cloud platforms (AWS, GCP, or Azure).

Familiarity with version control (Git).

Basic understanding of deep learning frameworks (e.g., TensorFlow, PyTorch).'''

In [20]:
chain = interview_prompt | llm | JsonOutputParser()
chain.invoke(job_desc)

{'interview_questions': [{'category': 'General & Behavioral',
   'questions': ['What motivated you to pursue a career as a Junior Data Scientist?',
    "What do you understand by 'data-driven insights' in the context of a business, and why is it important?",
    'How do you stay updated with the latest trends, tools, and research in data science?',
    "Describe a challenging problem you've encountered in a project or academic setting. How did you approach solving it, and what was the outcome?",
    'Where do you see yourself in the data science field in the next 3-5 years, and what steps are you taking to get there?']},
  {'category': 'Python & Data Manipulation (Pandas/NumPy)',
   'questions': ['What are the key differences between a Pandas DataFrame and a NumPy array? When would you choose one over the other for a specific data manipulation task?',
    'Describe various methods to handle missing values in a Pandas DataFrame. When would you use imputation versus dropping rows/columns

In [8]:
cover_letter_prompt = PromptTemplate.from_template('''You are an expert,proffesional cover letter writer . Your task is to create a proffesional one page cover letter .
                                                   Only use the content from job description and resume content and write a cover letter using this information only.
                                                   Job description:
                                                   {job_desc}
                                                   
                                                   Resume content:
                                                   {resume}''')